# Import packages

In [2]:
# LINC API
import hackathon_linc as lh
lh.init('92438482-5598-4e17-8b34-abe17aa8f598')

# For data handling
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta



    Welcome to the LINC Hackathon! Your token is now saved in the Console. 
    That means you don't need to carry that out when using the other functions
    as long as you don't close your console. 
    
    This function is only to be used once to authenticate your token.
    
    Happy coding!

    


# Load and format data


In [6]:
# Read historical data
df = pd.read_csv('historical_data/stockPrices_hourly.csv')

# Convert date column to datetime objects
df['gmtTime'] = pd.to_datetime(df['gmtTime'])

# Add price column
df['price'] = (df['askMedian'] + df['bidMedian']) / 2

# Sort values
df = df.sort_values(by=['symbol', 'gmtTime'])

###### Create DataFrame with daily values ######

# Aggregate daily open and close prices
df_daily = df.groupby(['symbol', df['gmtTime'].dt.date]).agg(
    openPrice=('price', 'first'),  # First price of the day (07:00)
    closePrice=('price', 'last'),  # Last price of the day (15:00)
    askVolume=('askVolume', 'sum'), # Sum ask volume over the entire day
    bidVolume=('bidVolume', 'sum') # Sum bid volume over the entire day
).reset_index()

# Rename the date column
df_daily.rename(columns={'gmtTime': 'date'}, inplace=True)

# Sort values
df_daily = df_daily.sort_values(by=['symbol', 'date'])

df.head()
latest_prices = df_daily.sort_values(by='date').groupby('symbol').last()['closePrice']
print(latest_prices)
print(df_daily['date'].unique().min())

symbol
STOCK1     209.030617
STOCK10    121.238517
STOCK2      44.995183
STOCK3      50.378800
STOCK4      24.419400
STOCK5       5.841417
STOCK6      78.758900
STOCK7      15.801067
STOCK8     108.852333
STOCK9      80.190133
Name: closePrice, dtype: float64
2015-04-22


# Calculate and plot hourly RSI

In [4]:
from functions.metrics import calculate_rsi
import holoviews as hv
import hvplot.pandas
hvplot.extension('bokeh')

# Get random symbol or choose symbol
symbol = df['symbol'].drop_duplicates().sample(n=1).values[0]
# symbol = 'STOCK5'

# Calculate RSI
periods = 8*14
df_rsi = calculate_rsi(df[df['symbol'] == symbol], periods)

# RSI lines
upper_line = 70
lower_line = 30
left_x = df_rsi['gmtTime'].min()  # Leftmost x-value

df_rsi
# Plot
# df_rsi.hvplot.line(
#     x='gmtTime',
#     y='RSI',
#     by='symbol',
#     height=600,
#     width=1000,
#     legend='right',
#     xlabel='Greenwich Mean Time',
#     ylabel='RSI',
#     title=f'Hourly RSI with period={periods} for {symbol}'
#     ) * \
#     hv.HLine(y=upper_line).opts(color='black', line_width=2, line_dash='dashed') * hv.Text(x=left_x, y=upper_line, text=f'RSI{upper_line}').opts(text_align='left', text_color='black', text_font_size='20pt') * \
#     hv.HLine(y=lower_line).opts(color='black', line_width=2, line_dash='dashed') * hv.Text(x=left_x, y=lower_line, text=f'RSI{lower_line}').opts(text_align='left', text_color='black', text_font_size='20pt')


In [ ]:
from functions.metrics import calculate_rsi
import holoviews as hv
import hvplot.pandas
hvplot.extension('bokeh')
from datetime import date, datetime

# Get random symbol or choose symbol
symbol = df['symbol'].drop_duplicates().sample(n=1).values[0]
# symbol = 'STOCK5'

# Calculate RSI
periods = 14
df_rsi = calculate_rsi(df_daily[df_daily['symbol'] == symbol], periods)

# RSI lines
upper_line = 70
lower_line = 30
left_x = datetime.combine(df_rsi['date'].min(), datetime.min.time())  # Leftmost x-value. Need to add 00:00:00 for it to match with data

# Plot
df_rsi.hvplot.line(
    x='date',
    y='RSI',
    by='symbol',
    height=600,
    width=1000,
    legend='right',
    xlabel='Greenwich Mean Time (Date only)',
    ylabel='RSI',
    title=f'Daily RSI with period={periods} for {symbol}'
    ) * \
    hv.HLine(y=upper_line).opts(color='black', line_width=2, line_dash='dashed') * hv.Text(x=left_x, y=upper_line, text=f'RSI{upper_line}').opts(text_align='left', text_color='black', text_font_size='20pt') * \
    hv.HLine(y=lower_line).opts(color='black', line_width=2, line_dash='dashed') * hv.Text(x=left_x, y=lower_line, text=f'RSI{lower_line}').opts(text_align='left', text_color='black', text_font_size='20pt')


# Mdified Markowitz portfolio optimization problem

## Calculate $\mu$ and $\Sigma$

In [63]:
# Calculate expected return per stock
df_return = df.groupby('symbol').agg(
    first_price=('price', 'first'),
    last_price=('price', 'last')
).reset_index()

# Assuming we know the number of years
years = 5  

df_return['returns'] = np.array((df_return['last_price'] / df_return['first_price']) ** (1 / years) - 1)
expected_returns = df_return['returns']

# Calculate daily returns per stock
df['returns'] = df.groupby('symbol')['price'].pct_change()

# Compute expected returns (mean return for each asset)
# expected_returns = df.groupby('symbol')['returns'].mean()

# Pivot the dataframe so that each column represents a stock's returns
returns_matrix = df.pivot(index='gmtTime', columns='symbol', values='returns')

# Compute the covariance matrix
cov_matrix = returns_matrix.cov()



## Optimize portfolio

In [64]:
from scipy.optimize import minimize

lambda_risk = 0.9   # Risk-aversion parameter (tune this)
z_alpha = 1.645     # Z-score for 95% confidence level
num_assets = len(expected_returns)

# Objective function: Maximize (expected return - risk-adjusted VaR)
def objective(w):
    w = np.array(w)
    portfolio_return = np.dot(w, expected_returns)  # w^T * mu
    portfolio_std = np.sqrt(np.dot(w.T, np.dot(cov_matrix, w)))  # sqrt(w^T * Sigma * w)
    var_risk = - (portfolio_return + z_alpha * portfolio_std)  # VaR term
    return -((1 + lambda_risk) * portfolio_return + lambda_risk * var_risk)  # Negative for minimization

# Constraints
constraints = [
    {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}  # Sum of weights = 1
]

# Bounds (no short-selling: weights between 0 and 1)
bounds = [(0, 1) for _ in range(num_assets)]

# Initial guess (equal allocation)
w0 = np.ones(num_assets) / num_assets

# Solve optimization problem
result = minimize(objective, w0, bounds=bounds, constraints=constraints)#, method='SLSQP')

# Optimal portfolio weights
optimal_weights = result.x

# Optimal portfolio returns
optimal_portfolio_return = np.dot(optimal_weights, expected_returns)

# Optimal Value at Risk
optimal_portfolio_std = np.sqrt(np.dot(optimal_weights.T, np.dot(cov_matrix, optimal_weights)))  # sqrt(w^T * Sigma * w)
optimal_var_risk = - (optimal_portfolio_return + z_alpha * optimal_portfolio_std)  # VaR term

print("Optimal Portfolio Weights:", optimal_weights)
print("Optimal Portfolio Return:", optimal_portfolio_return)
print("Optimal Portfolio VaR:", optimal_var_risk)



Optimal Portfolio Weights: [1.00000000e+00 0.00000000e+00 1.59594560e-16 0.00000000e+00
 3.69496100e-16 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 9.71445147e-17]
Optimal Portfolio Return: 0.22737867445643886
Optimal Portfolio VaR: -0.23563572349778097


In [65]:
z_alpha = 1.645  # Z-score for 95% confidence level
num_assets = len(expected_returns)

# Function to compute optimal portfolio return for a given lambda
def compute_optimal_return(lambda_risk):
    def objective(w):
        portfolio_return = np.dot(w, expected_returns)  # w^T * mu
        portfolio_std = np.sqrt(np.dot(w.T, np.dot(cov_matrix, w)))  # sqrt(w^T * Sigma * w)
        var_risk = - (portfolio_return + z_alpha * portfolio_std)  # VaR term
        return -((1 + lambda_risk) * portfolio_return + lambda_risk * var_risk)  # Negative for minimization

    # Constraints
    constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1}]  # Sum of weights = 1

    # Bounds (no short-selling)
    bounds = [(0, 1) for _ in range(num_assets)]

    # Initial guess (equal allocation)
    w0 = np.ones(num_assets) / num_assets

    # Solve optimization
    result = minimize(objective, w0, bounds=bounds, constraints=constraints, method='SLSQP')
    optimal_weights = result.x

    # Compute expected portfolio return for these weights
    optimal_return = np.dot(optimal_weights, expected_returns)
    
    return optimal_return

# Loop over different values of lambda
lambda_values = np.linspace(0, 2, 20)  # Vary lambda from 0 to 2 in 20 steps
expected_returns_list = [compute_optimal_return(l) for l in lambda_values]

# Create DataFrame for plotting
df_plot = pd.DataFrame({'lambda': lambda_values, 'expected_return': expected_returns_list})

# Plot using hvplot
df_plot.hvplot.line(
    x='lambda', 
    y='expected_return', 
    title="Expected Return vs. Risk Aversion (λ)",
    xlabel="Risk Aversion (λ)",
    ylabel="Expected Return (𝐄[Rₚ])"
    )


:Curve   [lambda]   (expected_return)

# Calculate and plot hourly moving average

In [ ]:
from functions.metrics import calculate_moving_average
import holoviews as hv
import hvplot.pandas
hvplot.extension('bokeh')

# Get random symbol or choose symbol
symbol = df['symbol'].drop_duplicates().sample(n=1).values[0]
# symbol = 'STOCK5'

# Calculate MA
df_ma50 = calculate_moving_average(df[df['symbol'] == symbol], 50)
df_ma150 = calculate_moving_average(df[df['symbol'] == symbol], 150)

# Plot
df_ma50.hvplot.line(
    x='gmtTime',
    y='MA_50',
    # by='symbol',
    height=600,
    width=1000,
    legend='right',
    xlabel='Greenwich Mean Time',
    ylabel='Price',
    title=f'Hourly moving averages and price for {symbol}',
    label='MA 50'
    ) * \
df_ma150.hvplot.line(
    x='gmtTime',
    y='MA_150',
    # by='symbol',
    label='MA 150'
    ) * \
df_ma150.hvplot.line(
    x='gmtTime',
    y='price',
    # by='symbol',
    label='Price'
    )

:Overlay
   .Curve.MA_50  :Curve   [gmtTime]   (MA_50)
   .Curve.MA_150 :Curve   [gmtTime]   (MA_150)
   .Curve.Price  :Curve   [gmtTime]   (price)

# Calculate and plot daily moving average

In [ ]:
from functions.metrics import calculate_moving_average
import holoviews as hv
import hvplot.pandas
hvplot.extension('bokeh')

# Get random symbol or choose symbol
# symbol = df['symbol'].drop_duplicates().sample(n=1).values[0]
# symbol = 'STOCK5'

# Calculate MA
df_ma50 = calculate_moving_average(df_daily[df_daily['symbol'] == symbol], 50)
df_ma150 = calculate_moving_average(df_daily[df_daily['symbol'] == symbol], 150)

# Plot
df_ma50.hvplot.line(
    x='date',
    y='MA_50',
    # by='symbol',
    height=600,
    width=1000,
    legend='right',
    xlabel='Greenwich Mean Time',
    ylabel='Price',
    title=f'Daily moving averages and price for {symbol}',
    label='MA 50'
    ) * \
df_ma150.hvplot.line(
    x='date',
    y='MA_150',
    # by='symbol',
    label='MA 150'
    ) * \
df_ma150.hvplot.line(
    x='date',
    y='close_price',
    # by='symbol',
    label='Price'
    )

:Overlay
   .Curve.MA_50  :Curve   [date]   (MA_50)
   .Curve.MA_150 :Curve   [date]   (MA_150)
   .Curve.Price  :Curve   [date]   (close_price)